<a href="https://colab.research.google.com/github/utkarshbelkhede/Financial_Dashboard/blob/master/notebooks/7_Final_DF_using_Custom_HuggingFace_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1. Setting up Working Directory

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Working Directory
import os

os.chdir('/content/drive/MyDrive/Documents/Market Intelligence')

#### 2. Config

In [ ]:
COMBINED_CSV = "./datasets/final_combined_dataset.csv"
SUMARIZED_MODEL = "sshleifer/distilbart-cnn-12-6"
CLASSIFIED_MODEL = "distilbert-base-uncased-finetuned-sst-2-english"

#### 3. Dataframe

In [ ]:
import pandas as pd
import re

In [ ]:
df = pd.read_csv(COMBINED_CSV)
df = df.iloc[:,1:]
df.head()

,Company,Reporting_Date,1,1A,1B,2,3,5,7,7A,8,9A
0,Meta,2021,Item 1. Business Overview Our mission is to g...,Item 1A. Risk Factors Certain factors may hav...,Item 1B. Unresolved Staff Comments None.,Item 2. Properties Our corporate headquarters...,Item 3. Legal Proceedings Beginning on March ...,Item 5. Market for Registrant's Common Equity...,Item 7. Management's Discussion and Analysis ...,Item 7A. Quantitative and Qualitative Disclos...,Item 8. Financial Statements and Supplementar...,Item 9A. Controls and Procedures Evaluation o...
1,Meta,2020,Item 1. Business Overview Our mission is to g...,Item 1A. Risk Factors Certain factors may hav...,Item 1B. Unresolved Staff Comments None.,Item 2. Properties Our corporate headquarter...,Item 3. Legal Proceedings Beginning on March ...,Item 5. Market for Registrant's Common Equity...,Item 7. Management's Discussion and Analysis ...,Item 7A. Quantitative and Qualitative Disclos...,Item 8. Financial Statements and Supplementar...,Item 9A. Controls and Procedures Evaluation o...
2,Meta,2019,Item 1. Business Overview Our mission is to g...,Item 1A. Risk Factors Certain factors may hav...,Item 1B. Unresolved Staff Comments None.,Item 2. Properties Our corporate headquarter...,Item 3. Legal Proceedings Beginning on March ...,Item 5. Market for Registrant's Common Equity...,Item 7. Management's Discussion and Analysis ...,Item 7A. Quantitative and Qualitative Disclos...,Item 8. Financial Statements and Supplementar...,Item 9A. Controls and Procedures Evaluation o...
3,Meta,2018,Item 1. Business Overview Our mission is to g...,Item 1A. Risk Factors Certain factors may hav...,Item 1B. Unresolved Staff Comments None.,Item 2. Properties Our corporate headquarter...,Item 3. Legal Proceedings Beginning on March ...,Item 5. Market for Registrant's Common Equity...,Item 7. Management's Discussion and Analysis ...,Item 7A. Quantitative and Qualitative Disclos...,Item 8. Financial Statements and Supplementar...,Item 9A. Controls and Procedures Evaluation o...
4,Meta,2017,Item 1. Business Overview Our mission is to g...,Item 1A. Risk Factors Certain factors may hav...,Item 1B. Unresolved Staff Comments None.,Item 2. Properties Our corporate headquarter...,Item 3. Legal Proceedings Beginning on May 22...,Item 5. Market for Registrant's Common Equity...,Item 7. Management's Discussion and Analysis ...,Item 7A. Quantitative and Qualitative Disclos...,Item 8. Financial Statements and Supplementar...,Item 9A. Controls and Procedures Evaluation o...


#### 4. Remove Duplicate


In [ ]:
def remove_duplicate_sentence(df, exclude_columns):
  """
  Removes duplicate sentences from columns.
  And Returns a Clean Dataframe.

  Parameters :
  -----------
  df: pandas.DataFrame - DataFrame on which you wish to perform Text-Preprocessing.
  exclude_columns: int - Number of columns to exclude. (Left-to-Right)
  """

  temp1 = df.iloc[:,:exclude_columns]
  arr = df.iloc[:,exclude_columns:].to_numpy()
  columns=list(df.iloc[:,exclude_columns:].columns)
  
  for row in range(len(arr)):
    for col in range(len(arr[0])):
      arr[row][col] = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", arr[row][col])

  for row in range(len(arr[0])):
    for col in range(len(arr)-1):
      arr[col][row], arr[col+1][row] = [i for i in arr[col][row] if i not in arr[col+1][row]], [j for j in arr[col+1][row] if j not in arr[col][row]]

  for row in range(len(arr)):
    for col in range(len(arr[0])):
      arr[row][col] = '. '.join(map(str, list(arr[row][col])))
  
  temp2 = pd.DataFrame(arr, columns=columns)

  return pd.concat([temp1, temp2], axis=1)

In [ ]:
def remove_duplicate_sentence_set(df, exclude_columns):
  """
  Removes duplicate sentences from columns.(Uses Set)
  And Returns a Clean Dataframe.

  Parameters :
  -----------
  df: pandas.DataFrame - DataFrame on which you wish to perform Text-Preprocessing.
  exclude_columns: int - Number of columns to exclude. (Left-to-Right)
  """

  temp1 = df.iloc[:,:exclude_columns]
  arr = df.iloc[:,exclude_columns:].to_numpy()
  columns=list(df.iloc[:,exclude_columns:].columns)

  text_set = {''}

  for row in range(len(arr)):
    for col in range(len(arr[0])):
      arr[row][col] = set(re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", arr[row][col]))
      text_set = text_set | arr[row][col]

  for row in range(len(arr)):
    for col in range(len(arr[0])):
      arr[row][col] = arr[row][col] & text_set
      arr[row][col] = '. '.join(map(str, list(arr[row][col])))
  
  temp2 = pd.DataFrame(arr, columns=columns)

  return pd.concat([temp1, temp2], axis=1)

In [ ]:
clean = remove_duplicate_sentence(df=df, exclude_columns=2)
clean

,Company,Reporting_Date,1,1A,1B,2,3,5,7,7A,8,9A
0,Meta,2021,". We had a global workforce of 71,970 employee...",". In the United States, there have been, and c...",. Item 1B.. Unresolved Staff Comments None.,". Item 2.. As of December 31, 2021, we owned ...","In addition, our platform and user data practi...",. The following graph shows a comparison of th...,". At our scale, it is very difficult to attrib...",. Equity investments accounted for under the e...,". In October 2021, we changed our corporate na...",Limitations on Effectiveness of Controls and P...
1,Meta,2020,. Facebook Reality Labs' augmented and virtual...,". At our scale, it is very difficult to attrib...",. Item 1B.. Unresolved Staff Comments None.,". In addition, we have offices in more than 80...","In addition, our platform and user data practi...",. The stock price performance of the following...,". At our scale, it is very difficult to attrib...",". Foreign currency losses of $129 million, $10...",. We have agreed to hold certain parties harml...,Limitations on Effectiveness of Controls and P...
2,Meta,2019,. We also help people discover and learn about...,". At our scale, it is very difficult to attrib...",. Item 1B.. Unresolved Staff Comments None.,. Item 2.. We also own 15 data centers global...,"In addition, our platform and user data practi...",". As of December 31, 2019 , there were 39 stoc...",". At our scale, it is very difficult to attrib...",. Quantitative and Qualitative Disclosures Abo...,. The aggregate intrinsic value of the options...,Limitations on Effectiveness of Controls and P...
3,Meta,2018,. The GDPR includes operational requirements f...,. We are subject to a variety of laws and regu...,. Item 1B.. Unresolved Staff Comments None.,". Item 2.. As of December 31, 2018 , we owned...",". We believe these lawsuits are without merit,...",". In December 2018, our board of directors aut...","Revenue in 2017 increased $13.02 billion, or 4...",We have experienced and will continue to exper...,. See Note 12 in these notes to the consolidat...,Limitations on Effectiveness of Controls and P...
4,Meta,2017,". We work directly with these advertisers, thr...",. We are subject to a variety of laws and regu...,. Item 1B.. Unresolved Staff Comments None.,". Item 2.. As such, we have excluded the squa...",. We believe we have multiple legal grounds to...,. Purchases of Equity Securities by the Issuer...,. General and administrative.. Although we bel...,We have experienced and will continue to exper...,. See Note 12 in these notes to the consolidat...,Limitations on Effectiveness of Controls and P...
5,Merck & Co.,2021,. The highly competitive animal health busines...,". Changes in government laws and regulations,...",. Item 1B.. Unresolved Staff Comments.. None.,. Item 2.. Mercks manufacturing operations ar...,The information called for by this Item is inc...,The principal market for trading of the Compan...,For details on the components of Other (income...,Item 7A.. Managements Discussion and Analysis...,The amount ultimately reclassified to Sales ma...,These statements are prepared in conformity wi...
6,Merck & Co.,2020,Research and Development Update The Company cu...,. As a result of the Companys dependence on ke...,. Item 1B.. Unresolved Staff Comments.. None.,A number of properties will be transferred to ...,The information called for by this Item is inc...,The principal market for trading of the Compan...,Impairments of Long-Lived Assets The Company a...,Item 7A.. Managements Discussion and Analysis...,"In our opinion, the consolidated financial sta...",These statements are prepared in conformity wi...
7,Merck & Co.,2019,. Competitive pressures have intensified as pr...,. As a result of the Companys dependence on ke...,. Item 1B.. Unresolved Staff Comments.. None.,. Item 2.. The Companys corporate headquarter...,The information called for by this Item is inc...,". As of January 31, 2020, there were approxima...","In February 2020, Merck confirmed tha

#### 5. Appling HuggingFace Model

In [ ]:
!pip install --q transformers -q

In [ ]:
from transformers import pipeline

In [ ]:
# Used to summarize text
def text_summarizer(summarizer, text):
  """
  Returns a Summarized string for text

  Parameters:
  -----------
  summarizer: transformers.pipelines - Huggingface Model object for Summarization.
  text: str - String to be summarized
  """

  sumr = summarizer(text, do_sample=False, truncation=True)
  
  return sumr[0][list(sumr[0].keys())[0]]

In [ ]:
# Used to analyze-sentiment text
def text_classifier(classifier, text):
  """
  Returns a Dictionary with Sentiment

  Parameters:
  -----------
  classifier: transformers.pipelines - Huggingface Model object for Classification.
  text: str - String to be classified
  """

  res = classifier(text, padding=True, truncation=True)
  
  return res[0]

In [ ]:
# Used to summarize text and returns a dataframe
def final_dataframe(text_function, model, df, exclude_columns):
  """
  Returns a Dataframe having applied Huggingface model

  Parameters:
  -----------
  text_function: Python function.
  model: transformers.pipelines - Huggingface Model object for Classification.
  df : pandas.DataFrame - DataFrame on which you wish to perform Text-Preprocessing.
  exclude_columns : int - Which column to exclude from Dataframe.
  """

  temp1 = df.iloc[:,:exclude_columns]
  arr = df.iloc[:,exclude_columns:].to_numpy()
  columns=list(df.iloc[:,exclude_columns:].columns)

  for row in range(len(arr)):
    for col in range(len(arr[0])):
      arr[row][col] = text_function(model, arr[row][col])

  temp2 = pd.DataFrame(arr, columns=columns)

  return pd.concat([temp1, temp2], axis=1)

In [ ]:
summarizer = pipeline('summarization', model=SUMARIZED_MODEL)

final = final_dataframe(text_summarizer, summarizer, clean, 2)
final.to_csv('./datasets/'+ SUMARIZED_MODEL.replace('/', '_') +'.csv', index=False)
pd.read_csv('./datasets/'+ SUMARIZED_MODEL.replace('/', '_') +'.csv').head()

In [ ]:
classifier = pipeline('sentiment-analysis', model=CLASSIFIED_MODEL)

final = final_dataframe(text_classifier, classifier, clean, 2)
final.to_csv('./datasets/'+ CLASSIFIED_MODEL.replace('/', '_') +'.csv', index=False)
pd.read_csv('./datasets/'+ CLASSIFIED_MODEL.replace('/', '_') +'.csv').head()

,Company,Reporting_Date,1,1A,1B,2,3,5,7,7A,8,9A
0,Meta,2021,"{'label': 'POSITIVE', 'score': 0.9124663472175...","{'label': 'NEGATIVE', 'score': 0.9969049096107...","{'label': 'NEGATIVE', 'score': 0.9989053010940...","{'label': 'POSITIVE', 'score': 0.9971873164176...","{'label': 'NEGATIVE', 'score': 0.9940243363380...","{'label': 'NEGATIVE', 'score': 0.9989150762557...","{'label': 'NEGATIVE', 'score': 0.9973219037055...","{'label': 'NEGATIVE', 'score': 0.9965571165084...","{'label': 'NEGATIVE', 'score': 0.9949929118156...","{'label': 'NEGATIVE', 'score': 0.9525972008705..."
1,Meta,2020,"{'label': 'POSITIVE', 'score': 0.5144032835960...","{'label': 'NEGATIVE', 'score': 0.9977102279663...","{'label': 'NEGATIVE', 'score': 0.9989053010940...","{'label': 'POSITIVE', 'score': 0.9943506717681...","{'label': 'NEGATIVE', 'score': 0.9984756112098...","{'label': 'NEGATIVE', 'score': 0.9985986351966...","{'label': 'NEGATIVE', 'score': 0.9937828183174...","{'label': 'NEGATIVE', 'score': 0.9981203675270...","{'label': 'NEGATIVE', 'score': 0.9926292896270...","{'label': 'NEGATIVE', 'score': 0.934410035610199}"
2,Meta,2019,"{'label': 'POSITIVE', 'score': 0.9474074244499...","{'label': 'NEGATIVE', 'score': 0.9979555606842...","{'label': 'NEGATIVE', 'score': 0.9989053010940...","{'label': 'POSITIVE', 'score': 0.9960851669311...","{'label': 'NEGATIVE', 'score': 0.9946576952934...","{'label': 'NEGATIVE', 'score': 0.9978011250495...","{'label': 'NEGATIVE', 'score': 0.9985656142234...","{'label': 'NEGATIVE', 'score': 0.9973078966140...","{'label': 'NEGATIVE', 'score': 0.9868752956390...","{'label': 'NEGATIVE', 'score': 0.9530308246612..."
3,Meta,2018,"{'label': 'POSITIVE', 'score': 0.980543315410614}","{'label': 'NEGATIVE', 'score': 0.9987450838088...","{'label': 'NEGATIVE', 'score': 0.9989053010940...","{'label': 'POSITIVE', 'score': 0.9918457865715...","{'label': 'NEGATIVE', 'score': 0.9970108270645...","{'label': 'NEGATIVE', 'score': 0.9985345602035...","{'label': 'NEGATIVE', 'score': 0.9932865500450...","{'label': 'NEGATIVE', 'score': 0.9960965514183...","{'label': 'NEGATIVE', 'score': 0.9974138736724...","{'label': 'NEGATIVE', 'score': 0.9812654852867..."
4,Meta,2017,"{'label': 'POSITIVE', 'score': 0.880610466003418}","{'label': 'NEGATIVE', 'score': 0.9986382126808...","{'label': 'NEGATIVE', 'score': 0.9989053010940...","{'label': 'NEGATIVE', 'score': 0.9125710129737...","{'label': 'NEGATIVE', 'score': 0.9956183433532...","{'label': 'NEGATIVE', 'score': 0.9982483386993...","{'label': 'NEGATIVE', 'score': 0.9915756583213...","{'label': 'NEGATIVE', 'score': 0.9975601434707...","{'label': 'NEGATIVE', 'score': 0.9949482679367...","{'label': 'NEGATIVE', 'score': 0.9071258306503..."
